- Want to represent problems as linear problems because of ease of explainability, ability to make decisions (proxy for decision making)
- Collinearity might over or under estimate the coefficient
- How to get rid of collinearity:
    - Remove redundant variables (linearly dependent combinations)
    - Use PCA
- MLE is used in deep learning models
- A data set represents a snapshot in time so we will never know the true function, MLE is one of the best ways to get to the population parameters
- Correlation between the indepdent variables and the error could be resolved by adding more data (variables or data points)
- Steps: Reduce the number of variables in the model (dimensionality reduction), increase the number of variables in the model, or collect more observations
- durben-watson test for autocorrelation

## **Demonstrating how model performance degrades with autocorrelation** using simulated data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [2]:
mu, sigma = 0, 5
x = np.random.uniform(40,80,100)
epsilon = np.random.normal(mu, sigma, 100)
y = 3 + 4*x + epsilon

In [3]:
model_reg = sm.OLS(y,x).fit()

In [ ]:
print(model_reg.summary()) # OLS assumes heteroskedacity and autocorrelation don't exist (this is what Covariance Type: nonrobust means)

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.999
Model:                            OLS   Adj. R-squared (uncentered):              0.999
Method:                 Least Squares   F-statistic:                          1.903e+05
Date:                Tue, 23 Sep 2025   Prob (F-statistic):                   2.19e-164
Time:                        20:45:21   Log-Likelihood:                         -314.61
No. Observations:                 100   AIC:                                      631.2
Df Residuals:                      99   BIC:                                      633.8
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [6]:
x_updated = sm.add_constant(x)
model_updated = sm.OLS(y, x_updated).fit()
print(model_updated.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.985
Model:                            OLS   Adj. R-squared:                  0.985
Method:                 Least Squares   F-statistic:                     6563.
Date:                Tue, 23 Sep 2025   Prob (F-statistic):           1.33e-91
Time:                        20:49:08   Log-Likelihood:                -311.72
No. Observations:                 100   AIC:                             627.4
Df Residuals:                      98   BIC:                             632.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.1336      2.958      2.412      0.0

In [7]:
epsilon[0] = np.random.normal(mu, sigma, 1)
for i in range(0,99):
  epsilon[i+1] = 0.4*epsilon[i] + 0.6*np.random.normal(mu, sigma, 1)

/var/folders/3n/b26hkpfd71x2jvpstpknt4ch0000gn/T/ipykernel_43560/2748598615.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  epsilon[0] = np.random.normal(mu, sigma, 1)
/var/folders/3n/b26hkpfd71x2jvpstpknt4ch0000gn/T/ipykernel_43560/2748598615.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  epsilon[i+1] = 0.4*epsilon[i] + 0.6*np.random.normal(mu, sigma, 1)


In [8]:
y = 3 + 4*x + epsilon

In [9]:
x_updated = sm.add_constant(x)
model_OLS = sm.OLS(y,x_updated).fit()
print(model_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                 2.585e+04
Date:                Tue, 23 Sep 2025   Prob (F-statistic):          1.52e-120
Time:                        20:51:26   Log-Likelihood:                -243.52
No. Observations:                 100   AIC:                             491.0
Df Residuals:                      98   BIC:                             496.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.2913      1.495      4.207      0.0

In [10]:
# a test for heteroskedascity
from statsmodels.stats.diagnostic import het_breuschpagan

In [ ]:
resid = model_OLS.resid
exog = model_OLS.model.exog
# gets design matrix

In [ ]:
lm_stat, lm_pvalue, f_stat, f_pvalue = het_breuschpagan(resid, exog) # pulling the stats out of the statistical test

In [ ]:
print(lm_pvalue) # p-value (fail to reject the null hypothesis) is high  so data is homoscedastic 

0.23051452234900272


In [15]:
print(f_pvalue)

0.2347341235477536


In [ ]:
# building general least squares (GLS) model
from scipy.linalg import toeplitz # diagonal matrix where all the variables from left to right are the same
# what a matrix would look like if it was autocorrelated


In [17]:
toeplitz(np.array([1,0.5,0,0,0,0,0,0]))

array([[1. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0.5, 1. , 0.5, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.5, 1. , 0.5, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.5, 1. , 0.5, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.5, 1. , 0.5, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.5, 1. , 0.5, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.5, 1. , 0.5],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.5, 1. ]])

In [18]:
rho = 0.4
cov_matrix = sigma**2 * toeplitz(np.append([1,rho], np.zeros(98)))

In [ ]:
print(sm.GLS(y,x_updated,cov_matrix).fit().summary()) # a better generalized model to handle autocorrelation
# will see a difference in the coefficient of the intercept variable which means there is more of a change in some default variable
# there is more default behaviour that was discovered (a truer relationship) when reducing the autocorrerlation between models

                            GLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.997
Model:                            GLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                 3.487e+04
Date:                Tue, 23 Sep 2025   Prob (F-statistic):          6.88e-127
Time:                        21:01:52   Log-Likelihood:                -237.46
No. Observations:                 100   AIC:                             478.9
Df Residuals:                      98   BIC:                             484.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.1986      1.330      3.909      0.0

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/refs/heads/main/Datasets/kc_house_data.csv') # housing data

In [ ]:
# expect autocorrelation because of supply/demand (previous observations do inform current observations), expect heteroscedascity 

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [26]:
x = df.iloc[:,3:] # removed the 2nd, 3rd, 4th columns from the data set
y = df['price']
# adding y-intercept
x = sm.add_constant(x)

In [27]:
model_houses = sm.OLS(y, x).fit() # baseline OLS model

In [28]:
print(model_houses.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.700
Model:                            OLS   Adj. R-squared:                  0.700
Method:                 Least Squares   F-statistic:                     2960.
Date:                Tue, 23 Sep 2025   Prob (F-statistic):               0.00
Time:                        21:10:02   Log-Likelihood:            -2.9460e+05
No. Observations:               21613   AIC:                         5.892e+05
Df Residuals:                   21595   BIC:                         5.894e+05
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const           6.69e+06   2.93e+06      2.282

- bedrooms and bathrooms are the biggest predictors of price
- bedrooms is negative so there is probably missing information, or capturing information we don't have a variable for, there is bias in the model, other issues within the data set
- take the bedrooms coefficient with a grain of salt
- model is bad don't use the coefficients to derive real insights
- there is lots of heteroskedascity
- there is multicollinearity somewhere

**research these notes**
Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.46e-19. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.

In [ ]:
resid = model_houses.resid
exog = model_houses.model.exog
lm_stat, lm_pvalue, f_stat, f_pvalue = het_breuschpagan(resid, exog)
print(f_pvalue) # means heteroskedascity exists, model isn't good

0.0


https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLSResults.get_robustcov_results.html

In [ ]:
# fitting a GLS model
print(model_houses.get_robustcov_results(cov_type='HC3').summary()) # level depends on how much we want to account for outliers (points with a high degree of influence)
# not much changed with this model

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.700
Model:                            OLS   Adj. R-squared:                  0.700
Method:                 Least Squares   F-statistic:                     1015.
Date:                Tue, 23 Sep 2025   Prob (F-statistic):               0.00
Time:                        21:20:25   Log-Likelihood:            -2.9460e+05
No. Observations:               21613   AIC:                         5.892e+05
Df Residuals:                   21595   BIC:                         5.894e+05
Df Model:                          17                                         
Covariance Type:                  HC3                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const           6.69e+06   2.96e+06      2.258

/opt/anaconda3/lib/python3.12/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 18, but rank is 17
  warnings.warn('covariance of constraints does not have full '


In [36]:
print(model_houses.get_robustcov_results(cov_type='HC1').summary())
# maybe we aren't able to deal with heteroscedascity in this way
# or maybe this model isn't fit for this data

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.700
Model:                            OLS   Adj. R-squared:                  0.700
Method:                 Least Squares   F-statistic:                     1020.
Date:                Tue, 23 Sep 2025   Prob (F-statistic):               0.00
Time:                        21:21:23   Log-Likelihood:            -2.9460e+05
No. Observations:               21613   AIC:                         5.892e+05
Df Residuals:                   21595   BIC:                         5.894e+05
Df Model:                          17                                         
Covariance Type:                  HC1                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const           6.69e+06   2.95e+06      2.266

/opt/anaconda3/lib/python3.12/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 18, but rank is 17
  warnings.warn('covariance of constraints does not have full '


In [37]:
# predict log-price instead of price to help control for variance
# few outliers (super influential) in the data set (could check with a bubble plot)
# could take out the outliers, could play around with the combination of independent variables. for this data set, it's probably linear models are not good choices for this data set